In [1]:
#разобраться с библиотеками
#считать страницу результатов авито
#считать все объявления этой страницы
#сформировать датафрейм из нужных данных
#считать следующие страницы
#попробовать забирать номера телефонов


In [2]:
#!pip install selenium
#!pip install webdriver-manager
#!pip install pytesseract
#!pip install tesserocr

In [3]:
import pytesseract
from PIL import Image
from time import sleep

In [4]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
import json

In [5]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_mac64.zip
[WDM] - Unpack archive /Users/zzema/.wdm/drivers/chromedriver/83.0.4103.39/mac64/chromedriver.zip


In [6]:
driver.get('https://www.avito.ru/moskva/mebel_i_interer/kompyuternye_stoly_i_kresla-ASgBAgICAURa0E8?cd=1&geoCoords=55.6005021%2C37.7254322&radius=5&user=1')
#webdriver.Chrome(executable_path='/path/to/chromedriver')
#button = driver.find_element_by_xpath('//button[@class="button button-origin button-origin_large button-origin_full-width recommendations-show-more js-recommendations-show-more"]').click()

In [7]:
requiredHtml = driver.page_source

In [9]:
#Получили массив товаров с первой страницы
table=soup.findAll("div", {"class": "item_table-wrapper"})
len(table)

50

In [10]:
table[0]

<div class="item_table-wrapper">
  <div class="description item_table-description">
    <div class="snippet-title-row"><h3 class="snippet-title" data-marker="item-title" data-shape="default"><a class="snippet-link" href="/moskva/mebel_i_interer/kompyuternyy_stul_1937307220" itemprop="url" target="_blank" title="Компьютерный стул в Москве">Компьютерный стул</a></h3></div>    <div class="snippet-price-row"><span class="snippet-price" data-marker="item-price" data-shape="default">
 500  ₽
  </span></div>              
 <div class="data">
                      <div><div class="item-address" data-shape="default"><div itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span class="item-address__radius" style="color: #5FA800">
 2 км
 </span>
   ·       <span class="item-address__string">
 Ореховый б-р
 </span><div class="item-address-georeferences"><span class="item-address-georeferences"><span class="item-address-georeferences-item"><span class="item-address-georefere

In [11]:
#Находит айди объявления для того чтобы по нему потом найти кнопку: "показать телефон"
def get_item_id(data):
    temp=data.find("span", {"class": "js-messenger-button"}).get('data-props')#text.strip()
    return json.loads(temp).get("itemId")
get_item_id(table[0])

1937307220

In [12]:
data1=[]
for ad in table[0:3]:
        try:
            div = ad.find('h3')
            title = div.text.strip()
        except:
            title = ''
        try:
            div = ad.find('h3')
            url = "https://avito.ru" + div.find('a').get('href')
        except:
            url = ''
        try:
            price = ad.find("span", {"class": "snippet-price"}).text.strip('  ₽\r\n')
        except:
            price = ''
        try:
            div = ad.find('span', class_='item-address-georeferences-item__content')
            metro = div.text.strip()
        except:
            metro = ''
        try:
            item_id = get_item_id(ad)
        except:
            item_id = ''
        data = {'title':title,
                'price':price,
                'metro':metro,
                'url':url,
                'item_id':item_id}
        data1.append(data)
        print(data1)
        sleep(0.5)

[{'title': 'Компьютерный стул', 'price': '500', 'metro': 'Красногвардейская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stul_1937307220', 'item_id': 1937307220}]
[{'title': 'Компьютерный стул', 'price': '500', 'metro': 'Красногвардейская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stul_1937307220', 'item_id': 1937307220}, {'title': 'Компьютерный стол', 'price': '1 500', 'metro': 'Красногвардейская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stol_1937563709', 'item_id': 1937563709}]
[{'title': 'Компьютерный стул', 'price': '500', 'metro': 'Красногвардейская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stul_1937307220', 'item_id': 1937307220}, {'title': 'Компьютерный стол', 'price': '1 500', 'metro': 'Красногвардейская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stol_1937563709', 'item_id': 1937563709}, {'title': 'Компьютерный стол школьный стол', 'price': '4 500', 'metro': 'Зябликово', 'url': '

In [13]:
import pandas as pd
data_pd=pd.DataFrame(data1)
data_pd.head()

,title,price,metro,url,item_id
0,Компьютерный стул,500,Красногвардейская,https://avito.ru/moskva/mebel_i_interer/kompyu...,1937307220
1,Компьютерный стол,1 500,Красногвардейская,https://avito.ru/moskva/mebel_i_interer/kompyu...,1937563709
2,Компьютерный стол школьный стол,4 500,Зябликово,https://avito.ru/moskva/mebel_i_interer/kompyu...,1939055124


In [14]:
#str1='//button[@class="js-item-extended-contacts\n item-extended-contacts\n \n button button-origin"]'
#button = driver.find_element_by_xpath('//button[@class="js-item-extended-contacts//n item-extended-contacts//n //n button button-origin"]')
#button.click()

#sleep(3)

#driver.save_screenshot('screenshot.png')

#        image = self.driver.find_element_by_xpath(
#            '//div[@class="item-phone-big-number js-item-phone-big-number"]//*')
#        location = image.location
#        size = image.size
#
#        self.crop(location, size)

In [18]:
str1='//button[@data-item-id="'+str(get_item_id(table[0]))+'"]'
str1

'//button[@data-item-id="1937307220"]'

In [20]:
temp1=driver.find_element_by_xpath(str1)
temp1

<selenium.webdriver.remote.webelement.WebElement (session="d8e1d873bb9c0c9e1e63e38503429f3b", element="44c90ff5-af80-49fc-ae81-399296099ede")>

In [21]:
temp1.click()

In [26]:
driver.save_screenshot('screenshot.png')

True

In [23]:
temp2=driver.find_elements_by_xpath('//img[@class="item-extended-phone"]')
location = temp2[0].location
size = temp2[0].size
location, size

({'x': 858, 'y': 486}, {'height': 29, 'width': 108})

In [24]:
len(temp2)

1

In [28]:
def crop(self, location, size):
    image = Image.open('screenshot.png')
    x = location['x']
    y = location['y']
    width = size['width']
    height = size['height']
    img=image.crop((x, y, x+width, y+height))
    img.save('tel.gif')
    print(pytesseract.image_to_string(img))
crop(driver, location, size)

8 926 759-04-52
